In [7]:
import sys
import numpy as np
import pandas as pd
import requests

def get_input(user_id):
    adress = 'http://35.180.178.243'

    req = requests.get(adress + '/reset?user_id=' + user_id)

    data = req.json()

    nb_users = data.pop('nb_users')
    nb_items = data.pop('nb_items')
    next_state = data.pop('next_state')
    columns = ['state_history', 'action_history', 'rewards_history']
    data_array = np.array([data.pop(c) for c in columns])
    df =  pd.DataFrame(data_array.T, columns=columns)
    return nb_users, nb_items, next_state, data_array

In [8]:
ID = 'VI2X71V0287S9F9B7SCU'

In [9]:
k = 10
t = []
for i in range(k):
    _, _, _ , data = get_input(ID)
    _, _, reward_history = data
    nb_reco = len(reward_history)
    good_reco = reward_history > 0
    nb_good_reco = np.sum(good_reco)
    t.append(100*nb_good_reco/nb_reco)

print('Distribution du taux de bonnes recommandations sur {} resets :'.format(k), t)
print('Taux moyen :', np.mean(t))
print('Ecart type :', np.std(t))

Distribution du taux de bonnes recommandations sur 10 resets : [28.5, 29.0, 28.5, 27.0, 29.0, 29.5, 25.0, 23.5, 34.0, 30.0]
Taux moyen : 28.4
Ecart type : 2.7092434368288134


In [10]:
nb_users, nb_items, next_state , data = get_input(ID)

state_history, action_history, reward_history = data
columns_names = ['user_id', 'item_id', 'item_price'] + ['metadata_' + str(i) for i in range(5)]
dfs = [pd.DataFrame(state_history[i], columns=columns_names) for i in range(len(state_history))]
df = pd.DataFrame(state_history[0], columns=columns_names)
df['hist_line'] = 0
df['action_item_id'] = state_history[0][action_history[0]][1] 
for i in range(1, len(state_history)):
    df_temp = pd.DataFrame(state_history[i], columns=columns_names)
    df_temp['hist_line'] = i
    df_temp['action_item_id'] = state_history[i][action_history[i]][1] 
    df = df.append(df_temp, ignore_index=True)

df_next_state = pd.DataFrame(next_state, columns=columns_names)
user_df = pd.DataFrame(np.arange(nb_users), columns=['user_id']).set_index('user_id')
item_df = pd.DataFrame(np.arange(nb_items), columns=['item_id']).set_index('item_id')
df.head()

print('Nombre d\'utilisateurs :', nb_users)
print('Nombre d\'items :', nb_items)

nb_reco = len(reward_history)
good_reco = reward_history > 0
nb_good_reco = np.sum(good_reco)
nb_bad_reco = nb_reco - nb_good_reco
print('Nombre de recommandations :', nb_reco)
print('Nombre de bonnes recommandations :', nb_good_reco)
print('Nombre de mauvaises recommandations :', nb_bad_reco)
print('Taux de bonnes recommandations : {}%'.format(100*nb_bad_reco/nb_reco))

Nombre d'utilisateurs : 100
Nombre d'items : 30
Nombre de recommandations : 200
Nombre de bonnes recommandations : 47
Nombre de mauvaises recommandations : 153
Taux de bonnes recommandations : 76.5%


## Utilisateurs/items non présents dans l'historique

In [11]:
print('Nombre d\'utilisateurs non présents', nb_users-df['user_id'].nunique())
print('Nombre d\'items non présents', nb_items-df['item_id'].nunique())

Nombre d'utilisateurs non présents 11
Nombre d'items non présents 0


## Metadonnées utilisateur

In [12]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('user_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  True
Metadata 1 :  True
Metadata 2 :  False
Metadata 3 :  False
Metadata 4 :  False


Les metadonnées 0 et 1 sont des métadonnées utilisateurs (d'après l'historique)

## Metadonnées produits

In [13]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('item_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  False
Metadata 1 :  False
Metadata 2 :  True
Metadata 3 :  True
Metadata 4 :  False


Les metadonnées 2 et 3 sont des métadonnées utilisateurs (d'après l'historique)

## Nombre de commandes par utilisateurs

In [14]:
print('Nombre de commande par utilisateur minimum :', df.groupby('user_id')[['hist_line']].count().min()[0]) # 0 si on compte les absents
print('Nombre de commande par utilisateur maximum :', df.groupby('user_id')[['hist_line']].count().max()[0])

Nombre de commande par utilisateur minimum : 30
Nombre de commande par utilisateur maximum : 146


## Diversité des achats (quantitative)

In [18]:
bought_hist_idx = np.arange(len(state_history))[reward_history > 0]
df_bought = df[df.hist_line.isin(bought_hist_idx)]
df_bought = df_bought[df_bought.action_item_id == df_bought.item_id]
diversity = df_bought.groupby('user_id')[['item_id']].nunique()
number = df_bought.groupby('user_id')[['item_id']].nunique()

In [16]:
print('Nombre d\'articles par utilisateur minimum :', df.groupby('user_id')[['item_id']].nunique().min()[0]) # 0 si on compte les absents

Nombre d'articles par utilisateur minimum : 30


## Rapport au prix

In [17]:
#df[['user_id', 'item_price', 'metadata_2', 'metadata_3', 'metadata_4']].groupby('user_id').describe()
print('Minimum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().min()[0])
print('Maximum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().max()[0])
print('Minimum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().min()[0])
print('Maximum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().max()[0])


Minimum de la dépense moyenne : 534.8587209026474
Maximum de la dépense moyenne : 559.3530752679507
Minimum de l'ecart type : 309.2160780474297
Maximum de l'ecart type : 319.8959855904223


## Apparition de nouveaux utilisateurs ?

In [ ]:
def send_pred(user_id, pred):
    adress = 'http://35.180.178.243'
    req = requests.get(adress + '/predict?user_id=' + user_id + '&recommended_item=' + str(pred))

    data = req.json()
    reward = data.pop('reward')
    state = data.pop('state')
    return reward, state